In [1]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install ipywidgets

In [41]:
jupyter nbextension enable --py widgetsnbextension


SyntaxError: invalid syntax (629580002.py, line 1)

In [28]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import dask.dataframe as dd
import duckdb

In [7]:
# Afficher le répertoire courant
print("Répertoire courant :", os.getcwd())

# Changer de répertoire (remplacez le chemin par celui de votre dossier)
os.chdir(r'C:\Users\11348337\Desktop\Airmiles')
print("Nouveau répertoire :", os.getcwd())

# Charger les fichiers
MEMBERS_DIM = pd.read_csv("MEMBERS_DIM.csv")
TRANSACTION_FACT = pd.read_csv("TRANSACTION_FACT.csv")
REWARD_FACT = pd.read_csv("REWARD_FACT.csv")

# Vérifier le chargement en affichant les premières lignes
print("MEMBERS_DIM:")
print(MEMBERS_DIM.head())

print("\nTRANSACTION_FACT:")
print(TRANSACTION_FACT.head())

print("\nREWARD_FACT:")
print(REWARD_FACT.head())

Répertoire courant : C:\Users\11348337\Desktop\Airmiles
Nouveau répertoire : C:\Users\11348337\Desktop\Airmiles
MEMBERS_DIM:
    MEMBER_ID   AGE GENDER LANGUAGE       TIER SMALL_BUSINESS_FLAG  \
0     1057621  80.0    NaN        E      Basic                   N   
1     1218019  50.0      F        E  Next Best                   N   
2    12125937  59.0      M        E      Basic                   N   
3    16489011  56.0      F        E  Next Best                   N   
4  1003721328  61.0    NaN        E      Basic                   N   

   TENURE_MONTHS PROV  CASH_BACK_POINTS_BALANCE  REWARD_POINTS_BALANCE  \
0            381   ON                        81                     13   
1            371   BC                         0                   3963   
2            282   ON                        41                    928   
3            243   ON                        35                   1677   
4            165   AB                         0                   1570   

  EMAILAB

In [8]:
# Sélection des colonnes numériques du DataFrame transaction_fact
numeric_cols = TRANSACTION_FACT.select_dtypes(include=['float64', 'int64']).columns
print("Statistiques descriptives des variables numériques dans transaction_fact :")
print(TRANSACTION_FACT[numeric_cols].describe())

Statistiques descriptives des variables numériques dans transaction_fact :
          MEMBER_ID      MONTH_ID  TRANSACTIONS  AMOUNT_SPENT  \
count  5.264811e+07  5.264811e+07  5.264811e+07  5.264811e+07   
mean   3.315873e+08  2.023059e+05  4.377825e+00  1.085478e+02   
std    4.671691e+08  8.167853e+01  1.004097e+02  7.294551e+02   
min    5.900000e+01  2.022010e+05  0.000000e+00 -3.378689e+05   
25%    5.545079e+06  2.022090e+05  1.000000e+00  1.034000e+01   
50%    1.322814e+07  2.023070e+05  2.000000e+00  2.536000e+01   
75%    1.002889e+09  2.024040e+05  4.000000e+00  6.660000e+01   
max    1.019999e+09  2.024120e+05  1.701800e+05  6.776738e+05   

       BASE_POINTS_EARNED  BONUS_POINTS_EARNED  REWARD_POINTS_EARNED  \
count        5.264811e+07         5.264811e+07          5.252999e+07   
mean         1.973932e+01         1.206518e+01          2.265567e+01   
std          1.335340e+02         1.678892e+02          1.652203e+02   
min         -6.757400e+04        -5.265000e+04     

In [39]:
# Vérifier le nombre de valeurs manquantes par colonne
missing_values = TRANSACTION_FACT.isnull().sum()
print("Nombre de valeurs manquantes par colonne dans transaction_fact:")
print(missing_values)

# Calculer le pourcentage de valeurs manquantes par colonne
missing_percentage = (TRANSACTION_FACT.isnull().sum() / len(TRANSACTION_FACT)) * 100
print("\nPourcentage de valeurs manquantes par colonne:")
print(missing_percentage)


Nombre de valeurs manquantes par colonne dans transaction_fact:
MEMBER_ID                        0
RETAILER                   5262326
MONTH_ID                         0
TRANSACTIONS                     0
AMOUNT_SPENT                     0
BASE_POINTS_EARNED               0
BONUS_POINTS_EARNED              0
REWARD_POINTS_EARNED        118127
CASH_BACK_POINTS_EARNED     118127
dtype: int64

Pourcentage de valeurs manquantes par colonne:
MEMBER_ID                  0.000000
RETAILER                   9.995279
MONTH_ID                   0.000000
TRANSACTIONS               0.000000
AMOUNT_SPENT               0.000000
BASE_POINTS_EARNED         0.000000
BONUS_POINTS_EARNED        0.000000
REWARD_POINTS_EARNED       0.224371
CASH_BACK_POINTS_EARNED    0.224371
dtype: float64


In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(TRANSACTION_FACT['TRANSACTIONS'], kde=True, color='skyblue')
plt.title("Distribution de la variable transactions")
plt.xlabel("transactions")
plt.ylabel("Fréquence")
plt.show()

In [41]:
# Calcul de la proportion de valeurs manquantes par MONTH_ID
# Pour chaque période, on compte le nombre de valeurs manquantes dans RETAILER
missing_by_month = TRANSACTION_FACT.groupby('MONTH_ID')['RETAILER'].apply(lambda x: x.isnull().sum())
# On compte également le nombre total de transactions par MONTH_ID
total_by_month = TRANSACTION_FACT.groupby('MONTH_ID')['RETAILER'].size()
# Calcul de la proportion pour chaque période
proportion_by_month = ((missing_by_month / total_by_month).reset_index(name='missing_proportion'))* 100
print(proportion_by_month)

    MONTH_ID  missing_proportion
0   20220100           11.364218
1   20220200           10.783359
2   20220300           11.363413
3   20220400           10.527526
4   20220500           10.432712
5   20220600           10.013141
6   20220700            9.310383
7   20220800           11.399091
8   20220900           10.909381
9   20221000           10.285144
10  20221100           11.041101
11  20221200           10.226698
12  20230100           10.919349
13  20230200           10.172592
14  20230300           10.445738
15  20230400           10.241286
16  20230500            9.565207
17  20230600            9.290535
18  20230700            9.051367
19  20230800           10.939451
20  20230900           10.406831
21  20231000            9.809190
22  20231100           10.887403
23  20231200            9.796747
24  20240100           10.224506
25  20240200            9.264537
26  20240300            9.665169
27  20240400            9.147087
28  20240500            8.946191
29  202406

In [11]:
TRANSACTION_FACT['RETAILER'] = TRANSACTION_FACT['RETAILER'].fillna("No Retailer")

In [45]:
# Vérifier le nombre de valeurs manquantes par colonne
missing_values = TRANSACTION_FACT.isnull().sum()
print("Nombre de valeurs manquantes par colonne dans transaction_fact:")
print(missing_values)

# Calculer le pourcentage de valeurs manquantes par colonne
missing_percentage = (TRANSACTION_FACT.isnull().sum() / len(TRANSACTION_FACT)) * 100
print("\nPourcentage de valeurs manquantes par colonne:")
print(missing_percentage)


Nombre de valeurs manquantes par colonne dans transaction_fact:
MEMBER_ID                       0
RETAILER                        0
MONTH_ID                        0
TRANSACTIONS                    0
AMOUNT_SPENT                    0
BASE_POINTS_EARNED              0
BONUS_POINTS_EARNED             0
REWARD_POINTS_EARNED       118127
CASH_BACK_POINTS_EARNED    118127
dtype: int64

Pourcentage de valeurs manquantes par colonne:
MEMBER_ID                  0.000000
RETAILER                   0.000000
MONTH_ID                   0.000000
TRANSACTIONS               0.000000
AMOUNT_SPENT               0.000000
BASE_POINTS_EARNED         0.000000
BONUS_POINTS_EARNED        0.000000
REWARD_POINTS_EARNED       0.224371
CASH_BACK_POINTS_EARNED    0.224371
dtype: float64


In [47]:
# Calcul de la proportion de valeurs manquantes par MONTH_ID
# Pour chaque période, on compte le nombre de valeurs manquantes dans REWARD_POINTS_EARNED
missing_by_month = TRANSACTION_FACT.groupby('MONTH_ID')['REWARD_POINTS_EARNED'].apply(lambda x: x.isnull().sum())
# On compte également le nombre total de transactions par MONTH_ID
total_by_month = TRANSACTION_FACT.groupby('MONTH_ID')['REWARD_POINTS_EARNED'].size()
# Calcul de la proportion pour chaque période
proportion_by_month = ((missing_by_month / total_by_month).reset_index(name='missing_proportion'))* 100
print(proportion_by_month)

    MONTH_ID  missing_proportion
0   20220100            0.226506
1   20220200            0.184993
2   20220300            0.187172
3   20220400            0.195113
4   20220500            0.222670
5   20220600            0.208314
6   20220700            0.337724
7   20220800            0.251085
8   20220900            0.214280
9   20221000            0.214539
10  20221100            0.336334
11  20221200            0.376111
12  20230100            0.221422
13  20230200            0.190040
14  20230300            0.198958
15  20230400            0.187205
16  20230500            0.214553
17  20230600            0.212108
18  20230700            0.212126
19  20230800            0.217575
20  20230900            0.218304
21  20231000            0.210894
22  20231100            0.225897
23  20231200            0.211574
24  20240100            0.216743
25  20240200            0.186581
26  20240300            0.209159
27  20240400            0.209929
28  20240500            0.216435
29  202406

In [49]:
# Calculer le pourcentage de valeurs manquantes de REWARDS_POINTS_EARNED pour chaque détaillant
missing_by_retailer = TRANSACTION_FACT.groupby('RETAILER')['REWARD_POINTS_EARNED']\
    .apply(lambda x: x.isnull().mean() * 100)\
    .reset_index(name='missing_percentage')

# Trier les résultats par pourcentage décroissant pour identifier rapidement les détaillants concernés
missing_by_retailer = missing_by_retailer.sort_values(by='missing_percentage', ascending=False)

print("Pourcentage de valeurs manquantes de REWARDS_POINTS_EARNED par détaillant :")
print(missing_by_retailer)


Pourcentage de valeurs manquantes de REWARDS_POINTS_EARNED par détaillant :
      RETAILER  missing_percentage
4   CATEGORY_5            1.674339
7   CATEGORY_8            1.516454
8  No Retailer            0.769318
5   CATEGORY_6            0.049945
0   CATEGORY_1            0.036981
6   CATEGORY_7            0.020244
2   CATEGORY_3            0.005988
3   CATEGORY_4            0.001941
1   CATEGORY_2            0.000753


In [51]:
# Créer des copies du DataFrame original pour chaque méthode d'imputation
df_median = TRANSACTION_FACT.copy()
df_mean   = TRANSACTION_FACT.copy()

# Imputation par la médiane : pour chaque RETAILER, on remplace les valeurs manquantes par la médiane du groupe
df_median['REWARD_POINTS_EARNED'] = df_median.groupby('RETAILER')['REWARD_POINTS_EARNED'] \
    .transform(lambda x: x.fillna(x.median()))

# Imputation par la moyenne : pour chaque RETAILER, on remplace les valeurs manquantes par la moyenne du groupe
df_mean['REWARD_POINTS_EARNED'] = df_mean.groupby('RETAILER')['REWARD_POINTS_EARNED'] \
    .transform(lambda x: x.fillna(x.mean()))

# Comparaison des statistiques descriptives
print("Statistiques descriptives après imputation par la médiane :")
print(df_median['REWARD_POINTS_EARNED'].describe())

print("\nStatistiques descriptives après imputation par la moyenne :")
print(df_mean['REWARD_POINTS_EARNED'].describe())



Statistiques descriptives après imputation par la médiane :
count    5.264811e+07
mean     2.260822e+01
std      1.650379e+02
min     -6.757400e+04
25%      0.000000e+00
50%      1.000000e+00
75%      9.000000e+00
max      2.850490e+05
Name: REWARD_POINTS_EARNED, dtype: float64

Statistiques descriptives après imputation par la moyenne :
count    5.264811e+07
mean     2.263929e+01
std      1.650356e+02
min     -6.757400e+04
25%      0.000000e+00
50%      2.000000e+00
75%      9.000000e+00
max      2.850490e+05
Name: REWARD_POINTS_EARNED, dtype: float64


In [13]:
# Imputation par la médiane des REWARDS_POINTS_EARNED pour chaque RETAILER
TRANSACTION_FACT['REWARD_POINTS_EARNED'] = TRANSACTION_FACT.groupby('RETAILER')['REWARD_POINTS_EARNED'] \
    .transform(lambda x: x.fillna(x.median()))


In [55]:
# Calculer le pourcentage de valeurs manquantes de REWARDS_POINTS_EARNED pour chaque détaillant
missing_by_retailer = TRANSACTION_FACT.groupby('RETAILER')['REWARD_POINTS_EARNED']\
    .apply(lambda x: x.isnull().mean() * 100)\
    .reset_index(name='missing_percentage')

# Trier les résultats par pourcentage décroissant pour identifier rapidement les détaillants concernés
missing_by_retailer = missing_by_retailer.sort_values(by='missing_percentage', ascending=False)

print("Pourcentage de valeurs manquantes de REWARDS_POINTS_EARNED par détaillant :")
print(missing_by_retailer)

Pourcentage de valeurs manquantes de REWARDS_POINTS_EARNED par détaillant :
      RETAILER  missing_percentage
0   CATEGORY_1                 0.0
1   CATEGORY_2                 0.0
2   CATEGORY_3                 0.0
3   CATEGORY_4                 0.0
4   CATEGORY_5                 0.0
5   CATEGORY_6                 0.0
6   CATEGORY_7                 0.0
7   CATEGORY_8                 0.0
8  No Retailer                 0.0


In [25]:
# Calculer le nombre de transactions par retailer
transaction_count = TRANSACTION_FACT['RETAILER'].value_counts()

# Calculer le pourcentage de transactions pour chaque retailer
transaction_percentage = (transaction_count / len(TRANSACTION_FACT)) * 100

# Convertir en DataFrame pour faciliter l'affichage
retailer_percentage_df = transaction_percentage.reset_index()
retailer_percentage_df.columns = ['RETAILER', 'Transaction_Percentage']

print(retailer_percentage_df)

      RETAILER  Transaction_Percentage
0   CATEGORY_2               33.042523
1   CATEGORY_3               17.318482
2   CATEGORY_1               12.223935
3  No Retailer                9.995279
4   CATEGORY_4                9.491394
5   CATEGORY_6                9.233733
6   CATEGORY_5                7.626932
7   CATEGORY_8                0.598584
8   CATEGORY_7                0.469137


In [57]:
# Calcul de la proportion de valeurs manquantes par MONTH_ID
# Pour chaque période, on compte le nombre de valeurs manquantes dans REWARD_POINTS_EARNED
missing_by_month = TRANSACTION_FACT.groupby('MONTH_ID')['CASH_BACK_POINTS_EARNED'].apply(lambda x: x.isnull().sum())
# On compte également le nombre total de transactions par MONTH_ID
total_by_month = TRANSACTION_FACT.groupby('MONTH_ID')['CASH_BACK_POINTS_EARNED'].size()
# Calcul de la proportion pour chaque période
proportion_by_month = ((missing_by_month / total_by_month).reset_index(name='missing_proportion'))* 100
print(proportion_by_month)

    MONTH_ID  missing_proportion
0   20220100            0.226506
1   20220200            0.184993
2   20220300            0.187172
3   20220400            0.195113
4   20220500            0.222670
5   20220600            0.208314
6   20220700            0.337724
7   20220800            0.251085
8   20220900            0.214280
9   20221000            0.214539
10  20221100            0.336334
11  20221200            0.376111
12  20230100            0.221422
13  20230200            0.190040
14  20230300            0.198958
15  20230400            0.187205
16  20230500            0.214553
17  20230600            0.212108
18  20230700            0.212126
19  20230800            0.217575
20  20230900            0.218304
21  20231000            0.210894
22  20231100            0.225897
23  20231200            0.211574
24  20240100            0.216743
25  20240200            0.186581
26  20240300            0.209159
27  20240400            0.209929
28  20240500            0.216435
29  202406

In [14]:
# Imputation par la médiane des CASH_BACK_POINTS_EARNED pour chaque RETAILER
TRANSACTION_FACT['CASH_BACK_POINTS_EARNED'] = TRANSACTION_FACT.groupby('RETAILER')['CASH_BACK_POINTS_EARNED'] \
    .transform(lambda x: x.fillna(x.median()))

In [12]:
# Calculer le pourcentage de valeurs manquantes de CASH_BACK_POINTS_EARNED pour chaque détaillant
missing_by_retailer = TRANSACTION_FACT.groupby('RETAILER')['CASH_BACK_POINTS_EARNED']\
    .apply(lambda x: x.isnull().mean() * 100)\
    .reset_index(name='missing_percentage')

# Trier les résultats par pourcentage décroissant pour identifier rapidement les détaillants concernés
missing_by_retailer = missing_by_retailer.sort_values(by='missing_percentage', ascending=False)

print("Pourcentage de valeurs manquantes de REWARDS_POINTS_EARNED par détaillant :")
print(missing_by_retailer)

Pourcentage de valeurs manquantes de REWARDS_POINTS_EARNED par détaillant :
      RETAILER  missing_percentage
0   CATEGORY_1                 0.0
1   CATEGORY_2                 0.0
2   CATEGORY_3                 0.0
3   CATEGORY_4                 0.0
4   CATEGORY_5                 0.0
5   CATEGORY_6                 0.0
6   CATEGORY_7                 0.0
7   CATEGORY_8                 0.0
8  No Retailer                 0.0


In [16]:
# Vérifier le nombre de valeurs manquantes par colonne
missing_values = TRANSACTION_FACT.isnull().sum()
print("Nombre de valeurs manquantes par colonne dans transaction_fact:")
print(missing_values)

# Calculer le pourcentage de valeurs manquantes par colonne
missing_percentage = (TRANSACTION_FACT.isnull().sum() / len(TRANSACTION_FACT)) * 100
print("\nPourcentage de valeurs manquantes par colonne:")
print(missing_percentage)


Nombre de valeurs manquantes par colonne dans transaction_fact:
MEMBER_ID                  0
RETAILER                   0
MONTH_ID                   0
TRANSACTIONS               0
AMOUNT_SPENT               0
BASE_POINTS_EARNED         0
BONUS_POINTS_EARNED        0
REWARD_POINTS_EARNED       0
CASH_BACK_POINTS_EARNED    0
dtype: int64

Pourcentage de valeurs manquantes par colonne:
MEMBER_ID                  0.0
RETAILER                   0.0
MONTH_ID                   0.0
TRANSACTIONS               0.0
AMOUNT_SPENT               0.0
BASE_POINTS_EARNED         0.0
BONUS_POINTS_EARNED        0.0
REWARD_POINTS_EARNED       0.0
CASH_BACK_POINTS_EARNED    0.0
dtype: float64


In [19]:
# Imaginons que votre DataFrame s'appelle TRANSACTION_FACT
# Sélectionner les colonnes souhaitées
df_transaction_extracted = TRANSACTION_FACT[[
    'MEMBER_ID',
    'RETAILER',
    'TRANSACTIONS',
    'AMOUNT_SPENT',
    'BASE_POINTS_EARNED',
    'BONUS_POINTS_EARNED',
    'REWARD_POINTS_EARNED',
    'CASH_BACK_POINTS_EARNED',
]]

# Afficher un aperçu des données extraites
print("DataFrame avec uniquement les colonnes nécessaires pour l'analyse :")
print(df_transaction_extracted.head())

DataFrame avec uniquement les colonnes nécessaires pour l'analyse :
   MEMBER_ID    RETAILER  TRANSACTIONS  AMOUNT_SPENT  BASE_POINTS_EARNED  \
0      19683  CATEGORY_3             2         19.95                   2   
1      19683  CATEGORY_3             2          8.62                   1   
2      19683  CATEGORY_4             1         72.65                  14   
3      19683  CATEGORY_2             1          9.11                   1   
4      19683  CATEGORY_2             1          8.63                   1   

   BONUS_POINTS_EARNED  REWARD_POINTS_EARNED  CASH_BACK_POINTS_EARNED  
0                    0                   0.0                      2.0  
1                    9                   0.0                     10.0  
2                    0                   0.0                     14.0  
3                    0                   0.0                      1.0  
4                    0                   0.0                      1.0  


In [17]:
# Sélection des colonnes numériques du DataFrame MEMBERS_DIM
numeric_cols = MEMBERS_DIM.select_dtypes(include=['float64', 'int64']).columns
print("Statistiques descriptives des variables numériques dans MEMBERS_DIM :")
print(MEMBERS_DIM[numeric_cols].describe())

Statistiques descriptives des variables numériques dans MEMBERS_DIM :
          MEMBER_ID           AGE  TENURE_MONTHS  CASH_BACK_POINTS_BALANCE  \
count  1.254299e+06  1.060394e+06   1.254299e+06              1.254299e+06   
mean   4.746504e+08  5.539343e+01   2.215653e+02              1.008349e+02   
std    4.987444e+08  2.338556e+01   1.058801e+02              1.007268e+03   
min    5.900000e+01 -7.099000e+03   0.000000e+00             -6.448000e+04   
25%    8.399090e+06  4.100000e+01   1.170000e+02              0.000000e+00   
50%    1.774989e+07  5.500000e+01   2.250000e+02              0.000000e+00   
75%    1.009366e+09  6.800000e+01   3.180000e+02              4.400000e+01   
max    1.019999e+09  2.023000e+03   3.960000e+02              4.671960e+05   

       REWARD_POINTS_BALANCE  
count           1.254299e+06  
mean            1.458798e+03  
std             5.185960e+03  
min            -1.584230e+05  
25%             0.000000e+00  
50%             3.500000e+01  
75%       

In [69]:
# Vérifier le nombre de valeurs manquantes par colonne
missing_values = MEMBERS_DIM.isnull().sum()
print("Nombre de valeurs manquantes par colonne dans MEMBERS_DIM:")
print(missing_values)

# Calculer le pourcentage de valeurs manquantes par colonne
missing_percentage = (MEMBERS_DIM.isnull().sum() / len(MEMBERS_DIM)) * 100
print("\nPourcentage de valeurs manquantes par colonne:")
print(missing_percentage)

Nombre de valeurs manquantes par colonne dans MEMBERS_DIM:
MEMBER_ID                        0
AGE                         193905
GENDER                      379617
LANGUAGE                       388
TIER                             0
SMALL_BUSINESS_FLAG              0
TENURE_MONTHS                    0
PROV                         81117
CASH_BACK_POINTS_BALANCE         0
REWARD_POINTS_BALANCE            0
EMAILABLE_FLAG              273105
MAILABLE_FLAG                    0
dtype: int64

Pourcentage de valeurs manquantes par colonne:
MEMBER_ID                    0.000000
AGE                         15.459233
GENDER                      30.265272
LANGUAGE                     0.030934
TIER                         0.000000
SMALL_BUSINESS_FLAG          0.000000
TENURE_MONTHS                0.000000
PROV                         6.467118
CASH_BACK_POINTS_BALANCE     0.000000
REWARD_POINTS_BALANCE        0.000000
EMAILABLE_FLAG              21.773517
MAILABLE_FLAG                0.000000
dtyp

In [67]:
# Vérifier le chargement en affichant les premières lignes
print("MEMBERS_DIM:")
print(MEMBERS_DIM.head())

MEMBERS_DIM:
    MEMBER_ID   AGE GENDER LANGUAGE       TIER SMALL_BUSINESS_FLAG  \
0     1057621  80.0    NaN        E      Basic                   N   
1     1218019  50.0      F        E  Next Best                   N   
2    12125937  59.0      M        E      Basic                   N   
3    16489011  56.0      F        E  Next Best                   N   
4  1003721328  61.0    NaN        E      Basic                   N   

   TENURE_MONTHS PROV  CASH_BACK_POINTS_BALANCE  REWARD_POINTS_BALANCE  \
0            381   ON                        81                     13   
1            371   BC                         0                   3963   
2            282   ON                        41                    928   
3            243   ON                        35                   1677   
4            165   AB                         0                   1570   

  EMAILABLE_FLAG MAILABLE_FLAG  
0              Y             Y  
1              N             Y  
2              N      

In [21]:
# Imaginons que votre DataFrame s'appelle members_dim
# Sélectionner les trois colonnes souhaitées
df_members_extracted = MEMBERS_DIM[['MEMBER_ID', 'CASH_BACK_POINTS_BALANCE', 'REWARD_POINTS_BALANCE']]

# Afficher un aperçu des données extraites
print("DataFrame avec uniquement les colonnes nécessaires pour l'analyse :")
print(df_members_extracted.head())

DataFrame avec uniquement les colonnes nécessaires pour l'analyse :
    MEMBER_ID  CASH_BACK_POINTS_BALANCE  REWARD_POINTS_BALANCE
0     1057621                        81                     13
1     1218019                         0                   3963
2    12125937                        41                    928
3    16489011                        35                   1677
4  1003721328                         0                   1570


In [73]:
# Sélection des colonnes numériques du DataFrame REWARD_FACT
numeric_cols = REWARD_FACT.select_dtypes(include=['float64', 'int64']).columns
print("Statistiques descriptives des variables numériques dans REWARD_FACT :")
print(REWARD_FACT[numeric_cols].describe())

Statistiques descriptives des variables numériques dans REWARD_FACT :
          MEMBER_ID      MONTH_ID   REDEMPTIONS  NUMBER_ITEMS_REDEEMED  \
count  2.221424e+06  2.221424e+06  2.221424e+06           2.221424e+06   
mean   3.142759e+08  2.023094e+05  1.214891e+00           2.104145e+00   
std    4.601991e+08  8.129172e+01  6.820943e-01           3.199830e+00   
min    5.900000e+01  2.022010e+05  1.000000e+00           0.000000e+00   
25%    5.930436e+06  2.022100e+05  1.000000e+00           1.000000e+00   
50%    1.272092e+07  2.023070e+05  1.000000e+00           1.000000e+00   
75%    1.002599e+09  2.024040e+05  1.000000e+00           2.000000e+00   
max    1.016553e+09  2.024120e+05  6.200000e+01           3.180000e+02   

       POINTS_REDEEMED  
count     2.221424e+06  
mean      3.239224e+02  
std       1.027995e+03  
min       0.000000e+00  
25%       1.000000e+02  
50%       1.000000e+02  
75%       2.000000e+02  
max       1.784400e+05  


In [75]:
# Vérifier le chargement en affichant les premières lignes
print("REWARD_FACT:")
print(REWARD_FACT.head())

REWARD_FACT:
   MEMBER_ID  MONTH_ID REWARDS_CATEGORY  REDEMPTIONS  NUMBER_ITEMS_REDEEMED  \
0    3919097    202305        Cash_Back            1                      2   
1    3919097    202411        Cash_Back            1                      1   
2    3919097    202210        Cash_Back            1                      2   
3    3919097    202402        Cash_Back            1                      1   
4    3940860    202309        Cash_Back            2                      2   

   POINTS_REDEEMED  
0              200  
1              100  
2              200  
3              100  
4              200  


In [21]:
# Vérifier le nombre de valeurs manquantes par colonne
missing_values = REWARD_FACT.isnull().sum()
print("Nombre de valeurs manquantes par colonne dans REWARD_FACT:")
print(missing_values)

# Calculer le pourcentage de valeurs manquantes par colonne
missing_percentage = (REWARD_FACT.isnull().sum() / len(REWARD_FACT)) * 100
print("\nPourcentage de valeurs manquantes par colonne:")
print(missing_percentage)

Nombre de valeurs manquantes par colonne dans REWARD_FACT:
MEMBER_ID                0
MONTH_ID                 0
REWARDS_CATEGORY         0
REDEMPTIONS              0
NUMBER_ITEMS_REDEEMED    0
POINTS_REDEEMED          0
dtype: int64

Pourcentage de valeurs manquantes par colonne:
MEMBER_ID                0.0
MONTH_ID                 0.0
REWARDS_CATEGORY         0.0
REDEMPTIONS              0.0
NUMBER_ITEMS_REDEEMED    0.0
POINTS_REDEEMED          0.0
dtype: float64


In [23]:
# Imaginons que votre DataFrame s'appelle reward_fact
# Sélectionner les colonnes souhaitées
df_reward_extracted = REWARD_FACT[[
    'MEMBER_ID',
    'REWARDS_CATEGORY',
    'REDEMPTIONS',
    'NUMBER_ITEMS_REDEEMED',
    'POINTS_REDEEMED'
]]

# Afficher un aperçu des données extraites
print("DataFrame avec uniquement les colonnes nécessaires pour l'analyse :")
print(df_reward_extracted.head())

DataFrame avec uniquement les colonnes nécessaires pour l'analyse :
   MEMBER_ID REWARDS_CATEGORY  REDEMPTIONS  NUMBER_ITEMS_REDEEMED  \
0    3919097        Cash_Back            1                      2   
1    3919097        Cash_Back            1                      1   
2    3919097        Cash_Back            1                      2   
3    3919097        Cash_Back            1                      1   
4    3940860        Cash_Back            2                      2   

   POINTS_REDEEMED  
0              200  
1              100  
2              200  
3              100  
4              200  


In [ ]:
# Conversion de pandas DataFrames en Dask DataFrames (avec un certain nombre de partitions)
dd_transaction = dd.from_pandas(df_transaction_extracted, npartitions=10)
dd_members     = dd.from_pandas(df_members_extracted, npartitions=10)
dd_reward      = dd.from_pandas(df_reward_extracted, npartitions=10)

# Jointure entre transaction et members
dd_merged = dd.merge(dd_transaction, dd_members, on='MEMBER_ID', how='inner')

# Jointure avec reward
dd_merged = dd.merge(dd_merged, dd_reward, on='MEMBER_ID', how='inner')

# Pour obtenir un pandas DataFrame en mémoire (si taille raisonnable)
df_merged = dd_merged.compute()

print(df_merged.shape)
print(df_merged.head())

In [ ]:
# Inscrivez vos DataFrames dans DuckDB comme tables temporaires
duckdb.register("df_trans", df_transaction_extracted)
duckdb.register("df_members", df_members_extracted)
duckdb.register("df_reward", df_reward_extracted)

# Requête SQL pour réaliser la jointure sur MEMBER_ID
query = """
SELECT t.*, m.*, r.*
FROM df_trans t
INNER JOIN df_members m ON t.MEMBER_ID = m.MEMBER_ID
INNER JOIN df_reward r ON t.MEMBER_ID = r.MEMBER_ID
"""

# Exécuter la requête et récupérer le résultat sous forme de DataFrame pandas
df_merged = duckdb.query(query).to_df()

# Affichage de quelques informations sur le DataFrame fusionné
print("Dimensions de la table fusionnée :", df_merged.shape)
print(df_merged.head())

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))